In [1]:
from ioMicro import *

In [34]:
import ioMicro
print(ioMicro.__file__)

C:\Users\Scope2\Scope2Scripts\Zane_Ying_Virus\ioMicro.py


### Map data

In [5]:
data_folder= r'S:\ZaneTrit_3_3_2023'

# map all the hybes
hybes =  glob.glob(data_folder+os.sep+'H*')
# map all the fovs
fovs = [os.path.basename(fl)for fl in glob.glob(hybes[0]+os.sep+'*.zarr')]
def get_Hi(fld): 
    try: return int(os.path.basename(fld)[1:]) 
    except: return -1
    
hybes = np.array(hybes)[np.argsort([get_Hi(hybe) for hybe in hybes ])]

In [6]:
def drift_correction(hybes,fov,save_file,iiref=None,redo=False,ssz=20):
    if iiref is None: iiref = len(hybes)//2
    dic_drift = {}
    if os.path.exists(save_file):
        dic_drift = pickle.load(open(save_file,'rb'))
    tags = [os.path.basename(hybe) for hybe in hybes]
    redo = ~np.all([tag in dic_drift for tag in tags])
    if redo:
        #Load the reference dapi image
        fl1 = hybes[iiref]+os.sep+fov
        im1 = read_im(fl1)
        ncols,sz,sx,sy = im1.shape
        im1dapi = np.array(im1[-1][(sz-ssz)//2:(sz+ssz)//2],dtype=np.float32)
        ###Perform drift

        for hybe in tqdm(hybes):
            fl2 = hybe+os.sep+fov
            tag = os.path.basename(hybe)
            if not (tag in dic_drift):
                im2 = read_im(fl2)
                ncols,sz,sx,sy = im1.shape
                

                im2dapi = np.array(im2[-1][(sz-ssz)//2:(sz+ssz)//2],dtype=np.float32)
                txyz,txyzs = get_txyz(im1dapi,im2dapi,sz_norm=30,sz = 500,nelems=5)
                dic_drift[tag]=[txyz,txyzs]
                #print(txyz,txyzs)
                pickle.dump(dic_drift,open(save_file,'wb'))

In [7]:
from scipy.signal import fftconvolve
def fit(hybe,fov,analysis_folder=r'S:\ZaneTrit_3_3_2023_analysis',icol=0,redo=False,hth = 50,plt_val=True):
    fl = hybe+os.sep+fov
    tag = os.path.basename(hybe)
    save_file=analysis_folder+os.sep+fov.split('.')[0]+'--'+tag+'_fits_icol'+str(icol)+'.npz'
    if not os.path.exists(save_file) or redo:
        im = read_im(fl)
        ncols,sz,sx,sy = im.shape
        #for icol in np.arange(ncols-1):
        imS = np.array(im[icol],dtype=np.float32) #GFP cy5 half
        #Convolve with gaussian 
        sz=5
        X = np.indices([2*sz+1]*3)-sz
        sigma=1.5
        gaussian = np.exp(-np.sum(X**2,axis=0)/2/sigma**2)
        im_conv = fftconvolve(imS,gaussian,mode='same')/np.sum(gaussian)
        #Subtract local background
        im_conv_ = norm_slice(im_conv,s=30)
        #fit the spots
        Xh = get_local_max(im_conv_,hth,im_raw=im_conv,delta=1,delta_fit=3)
        if plt_val:
            import napari
            v = napari.view_image(im_conv_)
            v.add_image(imS)
            h = Xh[:,-1]
            size = 5+np.clip(h/np.percentile(h,99.9),0,1)*10
            v.add_points(Xh[:,:3],size=size,face_color=[0,0,0,0],edge_color='y')
        np.savez_compressed(save_file,Xh=Xh)

In [87]:
for hybe in tqdm(hybes):
    for icol in range(2):
        fit(hybe, fovs[30],icol=icol,analysis_folder=r'S:\ZaneTrit_3_3_2023_analysis',redo=False,hth = 50,plt_val=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [31:24<00:00, 99.17s/it]


In [9]:

fit(hybes[4], fovs[30],icol=1,analysis_folder=r'S:\ZaneTrit_3_3_2023_analysis',redo=True,hth = 50,plt_val=True)


C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [7]:
def tag_to_hybe(tag):
    try:
        return int(tag[1:])
    except:
        return -1
class decoder_simple():
    def __init__(self,save_folder=r'S:\ZaneTrit_3_3_2023_analysis',fov='Conv_zscan__30',ncols=2,
                 hybe_bit_dic={(1, 1): 0, (1, 0): 1, (2, 0): 2, (2, 1): 3, (3, 1): 4, (3, 0): 5, (4, 1): 6, (4, 0): 7, (5, 0): 8, (5, 1): 9, (6, 1): 10, (6, 0): 11, (7, 1): 12, (7, 0): 13, (8, 0): 14, (8, 1): 15, (9, 1): 16, (9, 0): 17, (10, 1): 18, (10, 0): 19, (11, 0): 20, (11, 1): 21, (12, 1): 22, (12, 0): 23, (13, 1): 24, (13, 0): 25, (14, 1): 26, (14, 0): 27, (15, 1): 28, (15, 0): 29, (16, 1): 30, (16, 0): 31, (17, 1): 32, (17, 0): 33, (18, 1): 34, (18, 0): 35},
                chromatic_fl = 'chromatic_abberation_matrix_Scope2DC60xobj.npy'):
        self.save_folder = save_folder
        self.fov = fov
        self.decoded_fl = save_folder+os.sep+'decoded_'+fov.split('.')[0]+'.npz'
        self.drift_fl = save_folder+os.sep+fov.split('.')[0]+'_drift.pkl'
        self.ncols=ncols
        self.hybe_bit_dic=hybe_bit_dic
        self.chromatic_fl = self.save_folder+os.sep+chromatic_fl
    def get_XH(self):
        """Load in the fitted dots for each field of view.
        Apply chromatic abberation correction and drift correction and append corrected fitted data 
        int self.XH structure"""
        ncols = self.ncols
        drift_dic = pickle.load(open(self.drift_fl,'rb'))
        tags = list(drift_dic.keys())
        if not os.path.exists(self.chromatic_fl):
            self.m=None
        else:
            self.m = np.load(self.chromatic_fl)
        XH = []
        save_folder = self.save_folder
        fov = self.fov
        for tag in tqdm(tags):
            iH = tag_to_hybe(tag)
            for icol in range(ncols):
                #tag = os.path.basename(fld)#Conv_zscan__30--H_GFP_fits_icol0
                save_fl = save_folder+os.sep+fov.split('.')[0]+'--'+tag+'_fits_icol'+str(icol)+'.npz'
                Xh = np.load(save_fl)['Xh']
                tzxy = drift_dic[tag][0]
                m=None
                if icol==0: m=self.m
                Xh[:,:3] = apply_colorcor(Xh[:,:3],m=m)
                Xh[:,:3]+=tzxy# drift correction
                #chromatic abberation -to check
                bit = self.hybe_bit_dic.get((iH,icol),-1)
                icolR = np.array([[icol,bit]]*len(Xh))
                XH_ = np.concatenate([Xh,icolR],axis=-1)
                XH.extend(XH_)
        self.XH = np.array(XH)

In [8]:
dec = decoder_simple()
dec.get_XH()

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:07<00:00,  2.41it/s]


In [10]:
dec.XH.shape
zc,xc,yc ,bk,a,habs,correlation with PSF,h - brightness, color: 0-750, 1-647, bit

(2500148, 10)

In [13]:
np.unique(dec.XH[:,-1])

array([-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
       12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.,
       25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35.])

In [12]:
len('''zc,xc,yc,bk,a,habs,hn,h'''.split(','))

8

In [30]:
dec = decoder_simple()

In [32]:
m_mat = np.load(dec.chromatic_fl)

array([[ 9.41184748e-01, -3.10163088e-05,  7.84166315e-06,
         0.00000000e+00],
       [ 1.44952186e+00, -1.49976842e-04,  9.17708808e-04,
         0.00000000e+00],
       [-1.56259698e+00,  1.28744781e-03, -1.09815209e-06,
         0.00000000e+00]])

In [25]:
vals = ['Alexa750','Cy5']
bit_dic = {(ibit//2+1,icol):ibit for ibit,icol in enumerate([vals.index(e) for e in lst])}

In [27]:
print(bit_dic)

{(1, 1): 0, (1, 0): 1, (2, 0): 2, (2, 1): 3, (3, 1): 4, (3, 0): 5, (4, 1): 6, (4, 0): 7, (5, 0): 8, (5, 1): 9, (6, 1): 10, (6, 0): 11, (7, 1): 12, (7, 0): 13, (8, 0): 14, (8, 1): 15, (9, 1): 16, (9, 0): 17, (10, 1): 18, (10, 0): 19, (11, 0): 20, (11, 1): 21, (12, 1): 22, (12, 0): 23, (13, 1): 24, (13, 0): 25, (14, 1): 26, (14, 0): 27, (15, 1): 28, (15, 0): 29, (16, 1): 30, (16, 0): 31, (17, 1): 32, (17, 0): 33, (18, 1): 34, (18, 0): 35}


In [20]:
lst = """Cy5
Alexa750
Alexa750
Cy5
Cy5
Alexa750
Cy5
Alexa750
Alexa750
Cy5
Cy5
Alexa750
Cy5
Alexa750
Alexa750
Cy5
Cy5
Alexa750
Cy5
Alexa750
Alexa750
Cy5
Cy5
Alexa750
Cy5
Alexa750
Cy5
Alexa750
Cy5
Alexa750
Cy5
Alexa750
Cy5
Alexa750
Cy5
Alexa750""".split('\n')

In [14]:
dec = decoder_simple()
pickle.load(open(dec.drift_fl,'rb'))#Conv_zscan__30--H_GFP_fits_icol0.npz


{'H_GFP': [array([  0,   5, -93]),
  [array([  0,   5, -93], dtype=int64),
   array([  0,   5, -93], dtype=int64),
   array([  0,   5, -93], dtype=int64),
   array([  0,   4, -92], dtype=int64),
   array([  0,   6, -93], dtype=int64)]],
 'H1': [array([ 0, 99, 90]),
  [array([ 0, 98, 90], dtype=int64),
   array([ 0, 99, 91], dtype=int64),
   array([ 0, 99, 90], dtype=int64),
   array([ 0, 99, 90], dtype=int64),
   array([ 0, 98, 90], dtype=int64)]],
 'H2': [array([ 0, 22, 45]),
  [array([ 0, 22, 45], dtype=int64),
   array([ 0, 23, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64)]],
 'H3': [array([ 0, -7, 16]),
  [array([ 0, -7, 16], dtype=int64),
   array([ 0, -6, 16], dtype=int64),
   array([ 0, -6, 16], dtype=int64),
   array([ 0, -7, 16], dtype=int64),
   array([ 0, -7, 16], dtype=int64)]],
 'H4': [array([ 0, -9, 16]),
  [array([ 0, -9, 16], dtype=int64),
   array([ 0, -9, 16], dtype=int64),
   array([ 0

In [53]:
class decoder_simple():
    def __init__(self,save_folder,fov='Conv_zscan__001'):
        self.save_folder = save_folder
        self.fov = fov
        self.decoded_fl = save_folder+os.sep+'decoded_'+fov.split('.')[0]+'--'+set_+'.npz'
        self.drift_fl = save_folder+os.sep+'drift_'+fov.split('.')[0]+'--'+set_+'.pkl'
    def check_is_complete(self):
        if os.path.exists(self.decoded_fl):
            print("Completed")
            return 1
        if not os.path.exists(self.drift_fl):
            print("Did not detect fit files")
            return -1
        print("Not completed")
        return 0
        
    def get_fovs_sets(self):
        self.drift_fls = glob.glob(self.save_folder+os.sep+'drift_*.pkl')
        self.fov_sets = [os.path.basename(fl).replace('drift_','').replace('.pkl','').split('--')
                         for fl in self.drift_fls]
    def get_XH(self,fov,set_,ncols=3):
        self.set_ = set_
        save_folder = self.save_folder
        drift_fl = save_folder+os.sep+'drift_'+fov.split('.')[0]+'--'+set_+'.pkl'
        drifts,all_flds,fov = pickle.load(open(drift_fl,'rb'))
        self.drifts,self.all_flds,self.fov = drifts,all_flds,fov

        XH = []
        for iH in tqdm(np.arange(len(all_flds))):
            fld = all_flds[iH]
            if 'MER' in os.path.basename(fld):
                for icol in range(ncols):
                    tag = os.path.basename(fld)
                    save_fl = save_folder+os.sep+fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npy.npz'
                    if not os.path.exists(save_fl):save_fl = save_fl.replace('.npy','')
                    Xh = np.load(save_fl)['Xh']
                    tzxy = drifts[iH][0]
                    Xh[:,:3]+=tzxy# drift correction
                    ih = get_iH(fld) # get bit
                    bit = (ih-1)*3+icol
                    icolR = np.array([[icol,bit]]*len(Xh))
                    XH_ = np.concatenate([Xh,icolR],axis=-1)
                    XH.extend(XH_)
        self.XH = np.array(XH)
    def get_inters(self,dinstance_th=2,enforce_color=False):
        """Get an initial intersection of points and save in self.res"""
        res =[]
        if enforce_color:
            icols = self.XH[:,-2]
            XH = self.XH
            for icol in tqdm(np.unique(icols)):
                inds = np.where(icols==icol)[0]
                Xs = XH[inds,:3]
                Ts = cKDTree(Xs)
                res_ = Ts.query_ball_tree(Ts,dinstance_th)
                res += [inds[r] for r in res_]
        else:
            XH = self.XH
            Xs = XH[:,:3]
            Ts = cKDTree(Xs)
            res = Ts.query_ball_tree(Ts,dinstance_th)
        self.res = res
        
    def load_library(self,lib_fl = r'Z:\DCBBL1_3_2_2023\MERFISH_Analysis\codebook_0_New_DCBB-300_MERFISH_encoding_2_21_2023.csv',nblanks=-1):
        code_txt = np.array([ln.replace('\n','').split(',') for ln in open(lib_fl,'r') if ',' in ln])
        gns = code_txt[1:,0]
        code_01 = code_txt[1:,2:].astype(int)
        codes = np.array([np.where(cd)[0] for cd in code_01])
        codes_ = [list(np.sort(cd)) for cd in codes]
        nbits = np.max(codes)+1

        codes__ = codes_
        gns__ = list(gns)
        if nblanks>=0:
            ### get extrablanks
            from itertools import combinations
            X_codes = np.array((list(combinations(range(nbits),4))))
            X_code_01 = []
            for cd in X_codes:
                l_ = np.zeros(nbits)
                l_[cd] = 1
                X_code_01.append(l_)
            X_code_01 = np.array(X_code_01,dtype=int)
            from scipy.spatial.distance import cdist
            eblanks = np.where(np.min(cdist(code_01,X_code_01,metric='hamming'),0)>=4/float(nbits))[0]
            codes__ = codes_ + [list(e)for e in X_codes[eblanks]]
            gns__ = list(gns)+ ['blanke'+str(ign+1).zfill(4) for ign in range(len(eblanks))]
        
        bad_gns = np.array(['blank' in e for e in gns__])
        good_gns = np.where(~bad_gns)[0]
        bad_gns = np.where(bad_gns)[0]

        
        
        self.lib_fl = lib_fl ### name of coding library
        self.nbits = nbits ### number of bits
        self.gns_names = gns__  ### names of genes and blank codes
        self.bad_gns = bad_gns ### indices of the blank codes
        self.good_gns = good_gns ### indices of the good gene codes
        self.codes__ = codes__ ### final extended codes of form [bit1,bit2,bit3,bit4]
        self.codes_01 = code_01
        if nblanks>=0:
            self.codes_01 = np.concatenate([code_01,X_code_01[eblanks]],axis=0) ### final extended codes of form [0,1,0,0,1...]
        
        dic_bit_to_code = {}
        for icd,cd in enumerate(self.codes__): 
            for bit in cd:
                if bit not in dic_bit_to_code: dic_bit_to_code[bit]=[]
                dic_bit_to_code[bit].append(icd)
        self.dic_bit_to_code = dic_bit_to_code  ### a dictinary in which each bit is mapped to the inde of a code
    def get_icodes(self,nmin_bits=4,method = 'top4',redo=False,norm_brightness=None):    
        #### unfold res which is a list of list with clusters of loc.
        
        
        res = self.res

        import time
        start = time.time()
        res = [r for r in res if len(r)>=nmin_bits]
        #rlens = [len(r) for r in res]
        #edges = np.cumsum([0]+rlens)
        res_unfolder = np.array([r_ for r in res for r_ in r])
        #res0 = np.array([r[0] for r in res for r_ in r])
        ires = np.array([ir for ir,r in enumerate(res) for r_ in r])
        print("Unfolded molecules:",time.time()-start)

        ### get scores across bits
        import time
        start = time.time()
        RS = self.XH[:,-1].astype(int)
        brighness = self.XH[:,-3]
        brighness_n = brighness.copy()
        if norm_brightness is not None:
            colors = self.XH[:,norm_brightness]#self.XH[:,-1] for bits
            med_cols = {col: np.median(brighness[col==colors])for col in np.unique(colors)}
            for col in np.unique(colors):
                brighness_n[col==colors]=brighness[col==colors]/med_cols[col]
        scores = brighness_n[res_unfolder]
       
        bits_unfold = RS[res_unfolder]
        nbits = len(np.unique(RS))
        scores_bits = np.zeros([len(res),nbits])
        arg_scores = np.argsort(scores)
        scores_bits[ires[arg_scores],bits_unfold[arg_scores]]=scores[arg_scores]

        import time
        start = time.time()
        ### There are multiple avenues here: 
        #### nearest neighbors - slowest
        #### best dot product - reasonable and can return missing elements - medium speed
        #### find top 4 bits and call that a code - simplest and fastest


        if method == 'top4':
            codes = self.codes__
            vals = np.argsort(scores_bits,axis=-1)
            bcodes = np.sort(vals[:,-4:],axis=-1)
            base = [nbits**3,nbits**2,nbits**1,nbits**0]
            bcodes_b = np.sum(bcodes*base,axis=1)
            codes_b = np.sum(np.sort(codes,axis=-1)*base,axis=1)
            icodesN = np.zeros(len(bcodes_b),dtype=int)-1
            for icd,cd in enumerate(codes_b):
                icodesN[bcodes_b==cd]=icd
            bad = np.sum(scores_bits>0,axis=-1)<4
            icodesN[bad]=-1
            igood = np.where(icodesN>-1)[0]
            inds_spotsN =  np.zeros([len(res),nbits],dtype=int)-1
            inds_spotsN[ires[arg_scores],bits_unfold[arg_scores]]=res_unfolder[arg_scores]
            res_prunedN = np.array([inds_spotsN[imol][codes[icd]] for imol,icd in enumerate(icodesN) if icd>-1])
            scores_prunedN = np.array([scores_bits[imol][codes[icd]] for imol,icd in enumerate(icodesN) if icd>-1])
            icodesN = icodesN[igood]
        elif method == 'dot':
            icodesN = np.argmax(np.dot(scores_bits[:],self.codes_01.T),axis=-1)
            inds_spotsN =  np.zeros([len(res),nbits],dtype=int)-1
            inds_spotsN[ires[arg_scores],bits_unfold[arg_scores]]=res_unfolder[arg_scores]
            res_prunedN = np.array([inds_spotsN[imol][codes[icd]] for imol,icd in enumerate(icodesN) if icd>-1])
            scores_prunedN = np.array([scores_bits[imol][codes[icd]] for imol,icd in enumerate(icodesN) if icd>-1])

        print("Computed the decoding:",time.time()-start)

        import time
        start = time.time()

        mean_scores = np.mean(scores_prunedN,axis=-1)
        ordered_mols = np.argsort(mean_scores)[::-1]
        keep_mols = []
        visited = np.zeros(len(self.XH))
        for imol in tqdm(ordered_mols):
            r = np.array(res_prunedN[imol])
            r_ = r[r>=0]
            if np.all(visited[r_]==0):
                keep_mols.append(imol)
                visited[r_]=1
        keep_mols = np.array(keep_mols)
        self.scores_prunedN = scores_prunedN[keep_mols]
        self.res_prunedN = res_prunedN[keep_mols]
        self.icodesN = icodesN[keep_mols]
        print("Computed best unique assigment:",time.time()-start)
        
        XH_pruned = self.XH[self.res_prunedN]
        np.savez_compressed(self.decoded_fl,XH_pruned=XH_pruned,icodesN=self.icodesN,gns_names = np.array(self.gns_names))
        #XH_pruned -> 10000000 X 4 X 10 [z,x,y,bk...,corpsf,h,col,bit] 
        #icodesN -> 10000000 index of the decoded molecules in gns_names
        #gns_names
    def load_decoded(self):
        import time
        start= time.time()
        self.decoded_fl = self.save_folder+os.sep+'decoded_'+self.fov.split('.')[0]+'--'+self.set_+'.npz'
        self.XH_pruned = np.load(self.decoded_fl)['XH_pruned']
        self.icodesN = np.load(self.decoded_fl)['icodesN']
        self.gns_names = np.load(self.decoded_fl)['gns_names']
        print("Loaded decoded:",start-time.time())

(3, 11, 11, 11)

C:\Users\Scope2\anaconda3\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Points layer 'Points' at 0x245a3ea6730>

In [38]:
#v.add_points(Xh[:,:3])



<Points layer 'Points' at 0x2450d96c250>

In [39]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 1499-8C83

 Directory of C:\Users\Scope2\Scope2Scripts\Zane_Ying_Virus

03/27/2023  04:35 PM    <DIR>          .
03/27/2023  04:35 PM    <DIR>          ..
03/27/2023  04:02 PM    <DIR>          .ipynb_checkpoints
03/22/2023  06:14 PM            19,003 First_Exploration_of_data.ipynb
03/27/2023  04:35 PM             8,748 FitAndDriftCorrectData.ipynb
03/27/2023  03:55 PM            79,344 GFP2Color_ChromatiAberration.ipynb
03/22/2023  05:29 PM            88,169 ioMicro.py
03/22/2023  05:30 PM    <DIR>          __pycache__
               4 File(s)        195,264 bytes
               4 Dir(s)  17,668,148,068,352 bytes free


S:\ZaneTrit_3_3_2023_analysis\Conv_zscan__30_drift.pkl


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [03:51<00:00, 12.17s/it]


In [20]:
dic_drift = pickle.load(open(save_file,'rb'))

In [21]:
dic_drift

{'H_GFP': [array([  0,   5, -93]),
  [array([  0,   5, -93], dtype=int64),
   array([  0,   5, -93], dtype=int64),
   array([  0,   5, -93], dtype=int64),
   array([  0,   4, -92], dtype=int64),
   array([  0,   6, -93], dtype=int64)]],
 'H1': [array([ 0, 99, 90]),
  [array([ 0, 98, 90], dtype=int64),
   array([ 0, 99, 91], dtype=int64),
   array([ 0, 99, 90], dtype=int64),
   array([ 0, 99, 90], dtype=int64),
   array([ 0, 98, 90], dtype=int64)]],
 'H2': [array([ 0, 22, 45]),
  [array([ 0, 22, 45], dtype=int64),
   array([ 0, 23, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64),
   array([ 0, 22, 45], dtype=int64)]],
 'H3': [array([ 0, -7, 16]),
  [array([ 0, -7, 16], dtype=int64),
   array([ 0, -6, 16], dtype=int64),
   array([ 0, -6, 16], dtype=int64),
   array([ 0, -7, 16], dtype=int64),
   array([ 0, -7, 16], dtype=int64)]]}